In [1]:
import pickle
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv(r'C:\Users\Nitin Flavier\Desktop\Data Nexus\Data Science\ML_BootCamp\ML_Projects\Churn_Modeling_ANN\datasets\transformed_churn_data.csv')
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [3]:
X = df.drop('Exited',axis=1)
y = df['Exited']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.33,random_state=42)

In [4]:
scaler = StandardScaler() 

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [5]:
with open(r'..\pickle_files\scaler.pkl','wb') as file_obj:
    pickle.dump(scaler,file_obj)

### ANN implementation

In [6]:
import datetime
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard


In [7]:
model = Sequential([
    Dense(64,activation=LeakyReLU(alpha=0.1),input_shape=(X_train.shape[1],)), # HL-1
    Dense(32,activation=LeakyReLU(alpha=0.1)),  # HL-2
    Dense(1,activation='sigmoid')  # output layer
])


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
loss = tf.keras.losses.BinaryCrossentropy()
opt = tf.keras.optimizers.Adam(learning_rate=0.01)

# compile the model 
model.compile(optimizer=opt,loss=loss,metrics=['accuracy'])

In [10]:
# Setting up Tensorboard for training 
log_dir = r"..\logs\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [11]:
tensorflowCallback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [12]:
# lets say after a particular loss value my model is not improving we can
# do earlyStopping

# waits for 5 epochs if the loss does change much it will stop
earlyStoppingCallback = EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [13]:
# Train the model

model.fit(
    X_train,y_train,
    validation_data=(X_test,y_test),
    epochs=100,
    callbacks=[tensorflowCallback,earlyStoppingCallback]
)

Epoch 1/100


210/210 [==============================] - 1s 3ms/step - loss: 0.4038 - accuracy: 0.8261 - val_loss: 0.4140 - val_accuracy: 0.8012
Epoch 2/100
210/210 [==============================] - 0s 2ms/step - loss: 0.3633 - accuracy: 0.8479 - val_loss: 0.3400 - val_accuracy: 0.8609
Epoch 3/100
210/210 [==============================] - 0s 2ms/step - loss: 0.3593 - accuracy: 0.8494 - val_loss: 0.3435 - val_accuracy: 0.8633
Epoch 4/100
210/210 [==============================] - 0s 2ms/step - loss: 0.3513 - accuracy: 0.8539 - val_loss: 0.3474 - val_accuracy: 0.8676
Epoch 5/100
210/210 [==============================] - 0s 2ms/step - loss: 0.3475 - accuracy: 0.8575 - val_loss: 0.3384 - val_accuracy: 0.8627
Epoch 6/100
210/210 [==============================] - 0s 2ms/step - loss: 0.3443 - accuracy: 0.8576 - val_loss: 0.3492 - val_accuracy: 0.8658
Epoch 7/100
210/210 [==============================] - 0s 2ms/step - loss: 0.3421 - accuracy: 0.8554 - val_loss: 0.3388 - val_accuracy: 0.86

In [14]:
model.save(r'..\pickle_files\model.h5')

c:\Users\Nitin Flavier\Desktop\Data Nexus\Data Science\ML_BootCamp\ML_Projects\Churn_Modeling_ANN\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [15]:
# Loading tensorboard extension
%load_ext tensorboard

In [17]:
%tensorboard --logdir ../logs/fit

Reusing TensorBoard on port 6006 (pid 9124), started 0:00:04 ago. (Use '!kill 9124' to kill it.)